## Bit Decomposition

This notebook demonstrates the ability in Cicada to take shared secrets and decompose them into their bit representation. Note that this does add a dimension to the shape of the arrays passed in as arguments. For example, below we use the default field of 64 bits and pass in an array with shape (2,2). What is returned as an array of shares with shape (2,2,64). Each index in the (2,2) portion contains the 64 bits decomposition of the shared secret which was at that locale in the original operand. 

This operation is costly in terms of both communication and time. Also it is important to recall that our operands are shifted due to our fixed point numeric representation, so 1 is expected to have 16 leading zeros in our little-ednidan formatting to account for the 16 bits of fractional value precision.

In [1]:
# Copyright 2021 National Technology & Engineering Solutions
# of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,
# the U.S. Government retains certain rights in this software.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging

import numpy

import cicada.communicator
import cicada.encoder
import cicada.additive

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    secret = numpy.array([[0,1],[3,9]]) if communicator.rank == 0 else numpy.zeros((2,2))
    log.info(f"Player {communicator.rank} secret:\n{secret}", src=0)

    share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=secret.shape)
    
    decd_share = protocol.bit_decompose(share)
    revealed = (protocol.reveal(decd_share))
    log.info(f"Player {communicator.rank} revealed: {revealed}",src=0)
    log.info(f"Player {communicator.rank} revealed shape: {revealed.shape}",src=0)

main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:59523 from tcp://127.0.0.1:59523.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:59523 from tcp://127.0.0.1:59524.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:59523 from tcp://127.0.0.1:59525.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret:
[[0 1]
 [3 9]]
INFO:root:Player 0 revealed: [[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

[None, None, None]